<div align="center">

# **Limpieza**

</div>

## Librerias

In [20]:
library(dplyr)
library(tidyr)
source("../src/data/dividir_polizas_por_anio.R")
source("../src/data/ajustar_inflacion.R")

## Data

In [21]:
df_input <- read.csv("../data/input/Muestra_Siniestros_4.csv")
dim(df_input)
glimpse(df_input)

[1] 5000   18

Rows: 5,000
Columns: 18
$ Amparo          <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, "PERDI…
$ Amp             <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, "PPD",…
$ SumaDePagos     <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, 263295…
$ Modelo          <int> 2013, 2010, 2012, 2008, 2008, 2007, 2012, 2007, 2010, …
$ Color           <chr> "ROJO", "PLATA", "PLATA", "BLANCO", "VINO TINTO", "PLA…
$ Carroceria      <chr> "SEDAN", "HATCHBACK", "SEDAN", "SEDAN", "SEDAN", "HATC…
$ MARCA           <chr> "HYUNDAI", "HYUNDAI", "HYUNDAI", "HYUNDAI", "HYUNDAI",…
$ Referencia1     <chr> "VELOSTER", "ATOS [2]", "ACCENT i25", "ATOS [2]", "ACC…
$ Referencia2     <chr> "COUPE", "PRIME", "1.4L", "PRIME", "GLS", "SANTRO", "G…
$ CLASE_FASECOLDA <chr> "AUTOMOVIL", "AUTOMOVIL", "AUTOMOVIL", "AUTOMOVIL TAXI…
$ TIPO_VEHICULO   <chr> "Livianos", "Livianos", "Livianos", "Livianos", "Livia…
$ SERVICIO        <chr> "Particular", "Particular", "Particular", "Particular"…
$ Sexo_Aseg     

In [22]:
unique(df_input$MARCA)

[1] "HYUNDAI"

In [23]:
nrow(df_input)

[1] 5000

In [24]:
colSums(is.na(df_input))

Amparo             Amp     SumaDePagos          Modelo           Color 
           4320            4320            4320               0               0 
     Carroceria           MARCA     Referencia1     Referencia2 CLASE_FASECOLDA 
              0               0               0               0               0 
  TIPO_VEHICULO        SERVICIO       Sexo_Aseg            Edad           Desde 
              0               0             625               0               0 
          Hasta    Vr_Comercial            Pago 
              0               0               0

## Preparacion de los datos

Nombres de variables

In [25]:
df_input$Color <- gsub("CHAMPA�A", "CHAMPAÑA", df_input$Color)
df_input$Amparo <- gsub("DA�OS", "DAÑOS", df_input$Amparo)

Nulos en Amp

In [26]:
df_input <- df_input %>%
  mutate(
    # Convertir NA a "No aplica"
    Amparo = ifelse(is.na(Amparo), "No aplica", Amparo),
    Amp = ifelse(is.na(Amp), "No aplica", Amp)
    )

Nulos en SumaDePagos

In [27]:
sort(unique(df_input$SumaDePagos))

[1]        0.0    26248.0    60000.0    69257.0    70000.0    80000.0
  [7]    83564.0   107000.0   110000.0   112500.0   125000.0   136045.0
 [13]   169000.0   170010.0   174544.0   176850.0   180000.0   189000.0
 [19]   260626.0   280163.0   285409.0   341480.0   357077.0   368906.0
 [25]   392440.0   392598.0   416435.0   417257.4   439700.0   448896.0
 [31]   449209.0   470000.0   494500.0   506378.0   513041.0   549648.0
 [37]   566700.0   577863.0   586600.0   589329.0   590774.0   607548.0
 [43]   614649.0   618453.0   649457.0   650000.0   659637.0   661064.0
 [49]   670567.0   671791.0   700000.0   712833.0   715614.0   716849.0
 [55]   730000.0   739659.5   740191.0   745349.0   752312.0   753487.0
 [61]   755600.0   780963.0   783800.0   784693.0   794499.0   825000.0
 [67]   825576.0   826419.0   835502.0   836368.0   852644.0   880360.0
 [73]   881701.0   884250.0   910000.0   925551.0   930564.0   939324.0
 [79]   950337.0   958692.0   973574.0   975296.0   977285.0   978133.8
 [85]   990000.0   991244.0   993320.0  1012745.0  1014267.0  1024899.0
 [91]  1030035.0  1048789.0  1048894.0  1060462.0  1061361.0  1064840.0
 [97]  1091656.0  1101346.0  1102693.0  1105819.0  1140190.0  1142079.0
[103]  1142986.0  1145801.0  1159312.0  1160356.0  1169648.0  1178869.0
[109]  1181614.0  1192652.0  1200000.0  1200192.0  1200604.0  1212002.0
[115]  1214039.0  1217000.0  1217367.0  1232252.0  1238335.0  1249181.0
[121]  1251702.0  1267555.0  1268916.0  1270787.1  1272179.0  1311097.0
[127]  1319084.0  1333859.0  1340186.0  1341314.0  1356422.0  1360843.0
[133]  1369500.0  1384891.0  1391720.0  1398370.0  1400000.0  1408675.0
[139]  1408842.0  1433400.0  1437703.0  1458638.0  1460496.0  1473483.0
[145]  1476756.0  1486151.0  1507264.0  1508000.0  1518880.0  1542625.0
[151]  1543881.0  1555945.0  1561025.0  1582952.0  1588828.0  1605695.0
[157]  1610107.0  1619998.0  1664038.0  1667862.0  1668459.0  1671349.0
[163]  1672151.0  1674936.0  1680000.0  1683205.0  1695277.0  1696871.9
[169]  1699904.0  1713360.0  1730764.0  1748203.0  1750888.0  1776094.0
[175]  1780917.2  1782924.0  1800000.0  1813436.0  1818856.0  1860839.0
[181]  1898317.0  1915817.0  1916318.0  1917339.0  1929575.0  1930375.0
[187]  1941288.0  1944502.0  1945571.0  1973535.0  1981348.0  1988784.0
[193]  1993871.0  2005282.0  2007719.0  2011959.0  2014788.0  2023823.0
[199]  2033227.0  2034439.0  2045370.0  2051869.0  2055471.0  2106000.0
[205]  2149110.0  2164200.0  2165905.0  2196687.0  2201099.4  2218069.0
[211]  2232785.0  2253050.0  2258496.0  2259212.0  2269678.0  2280489.0
[217]  2313334.0  2349555.0  2351684.0  2362927.0  2363428.0  2372723.0
[223]  2382921.0  2388797.0  2444146.0  2461050.0  2478747.0  2502467.0
[229]  2510170.0  2510591.0  2543890.0  2552804.0  2566862.0  2570485.0
[235]  2573989.0  2579112.0  2598403.0  2611404.0  2615391.0  2624459.0
[241]  2631123.0  2632957.0  2668118.0  2669499.4  2719829.0  2720821.0
[247]  2749222.0  2767108.0  2793013.0  2815057.0  2830478.0  2832849.0
[253]  2837428.0  2843407.0  2858185.0  2863824.0  2878939.0  2908142.0
[259]  2912640.0  2933002.3  2933400.0  2941068.0  2942736.0  2955946.0
[265]  2961646.0  2978301.0  2983269.0  2988531.0  3039741.0  3059736.0
[271]  3080625.0  3084225.0  3094734.0  3124576.0  3133980.0  3136481.0
[277]  3150000.0  3157366.0  3167455.0  3191091.0  3202503.0  3202962.0
[283]  3221794.0  3233175.0  3238754.0  3246366.0  3257837.0  3258600.0
[289]  3261432.0  3277395.0  3305604.0  3329053.0  3348824.0  3352567.0
[295]  3363328.0  3370954.0  3375045.0  3381472.0  3388265.0  3484887.0
[301]  3494837.0  3500000.0  3520149.0  3541975.0  3549665.0  3551268.0
[307]  3559185.0  3568119.0  3575208.0  3580000.0  3583185.0  3593878.0
[313]  3597475.0  3604485.0  3620332.0  3633300.0  3639276.0  3656473.0
[319]  3669705.0  3688106.0  3704003.0  3716816.0  3774274.0  3822796.2
[325]  3834700.0  3850017.0  3855000.0  3862903.0  3867842.0  3884011.0
[331]  3884268.0  3899393.0  3900609.0  39

In [28]:
pago_minimo <- 100000
df_input <- df_input %>% mutate(
    SumaDePagos = case_when(
      is.na(SumaDePagos) ~ 0,
      SumaDePagos < pago_minimo ~ 0,
      TRUE ~ SumaDePagos)
)

df_input <- df_input %>% mutate(
  Accidentado = ifelse(SumaDePagos >= pago_minimo, 1, 0)
)

Nulos en genero

In [29]:
df_input <- df_input %>% mutate(
    Sexo_Aseg = ifelse(is.na(Sexo_Aseg), "No aplica", Sexo_Aseg)
)

Segmentando por año

In [30]:
df_input <- dividir_polizas_por_anio(df_input)

Calculo de exposicion

In [31]:
df_input$exposicion <- as.numeric(df_input$Hasta - df_input$Desde)

Ajustando a inflacion de 2015

In [32]:
df_input <- ajustar_inflacion_2015(df_input)

Variables que sobran

In [33]:
df_input[c("Amp", "MARCA", "Pago", "Desde", "Hasta")] <- NULL

Ajustando valor comercial

In [34]:
vr_minimo <- 4000000
df_input <- df_input %>% mutate(
    Vr_Comercial = case_when(
      is.na(Vr_Comercial) ~ 0,
      Vr_Comercial < vr_minimo ~ 0,
      TRUE ~ Vr_Comercial)
)

Ajustando edad

In [35]:
edad_minima <- 18
edad_maxima <- 90

condicion_edad <- df_input$Edad >= edad_minima & df_input$Edad <= edad_maxima

df_no_edades <- df_input[!condicion_edad, ]
df_no_edades$Edad <- NULL

df_input <- df_input[condicion_edad, ]

Partir el dataframe en las 3 partes

In [36]:
df_completo <- df_input[!(df_input$Vr_Comercial == 0 | df_input$Sexo_Aseg == "No aplica"),]

df_no_vr <- df_input[(df_input$Vr_Comercial == 0), ]
df_no_vr$Vr_Comercial <- NULL

Resultados

In [37]:
colSums(is.na(df_input))

Amparo     SumaDePagos          Modelo           Color      Carroceria 
              0               0               0               0               0 
    Referencia1     Referencia2 CLASE_FASECOLDA   TIPO_VEHICULO        SERVICIO 
              0               0               0               0               0 
      Sexo_Aseg            Edad    Vr_Comercial     Accidentado      exposicion 
              0               0               0               0               0

In [38]:
summary(df_input)

    Amparo           SumaDePagos           Modelo        Color          
 Length:7245        Min.   :       0   Min.   :1993   Length:7245       
 Class :character   1st Qu.:       0   1st Qu.:2008   Class :character  
 Mode  :character   Median :       0   Median :2010   Mode  :character  
                    Mean   :  283070   Mean   :2009                     
                    3rd Qu.:       0   3rd Qu.:2012                     
                    Max.   :35206209   Max.   :2013                     
  Carroceria        Referencia1        Referencia2        CLASE_FASECOLDA   
 Length:7245        Length:7245        Length:7245        Length:7245       
 Class :character   Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character   Mode  :character  
                                                                            
                                                                            
                           

Exportando los resultados

In [39]:
write.csv(df_input, "../data/processed/datos_limpios.csv", row.names = FALSE)
write.csv(df_no_edades, "../data/processed/datos_no_edades.csv", row.names = FALSE)
write.csv(df_no_vr, "../data/processed/datos_no_vr.csv", row.names = FALSE)